In [ ]:
%pwd

In [ ]:
import sys
sys.path.append("E:\\Github Projects\\Retinal_Blood_Vessels_Segmentation")

In [ ]:
import cv2
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.under_sampling import RandomUnderSampler, NearMiss

from scripts.prepare_data import create_dataset_from_directory

In [ ]:
X, y = create_dataset_from_directory(dir="../images/CHASE", channel=1, window_shape=(5, 5), pad=True, padding=(2, 2))

In [ ]:
X.shape, y.shape, np.unique(y, return_counts=True)

In [ ]:
undersampler = RandomUnderSampler(random_state=42)

X_undersampled, y_undersampled = undersampler.fit_resample(X, y)

In [ ]:
X_undersampled.shape, y_undersampled.shape, np.unique(y_undersampled, return_counts=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_undersampled, y_undersampled, train_size=0.8, random_state=42)

X_train, X_test = X_train/255, X_test/255
y_train, y_test = (y_train/255).astype(int), (y_test/255).astype(int)

X_train, y_train = resample(X_train, y_train, random_state=42, stratify=y_train, n_samples=200000, replace=False)

In [ ]:
X_train.shape, y_train.shape, np.unique(y_train, return_counts=True)

In [ ]:
pipeline = Pipeline(steps=[('clf', RandomForestClassifier())])

params = [
    {
        "clf": [RandomForestClassifier()],
        "clf__criterion": ["gini", "entropy"],
        "clf__max_depth": np.arange(4, 13, 2),
        "clf__n_estimators": np.arange(10, 81, 10),
    },
    {
        "clf": [SVC()],
        "clf__C": [0.1, 1, 10, 100],
        "clf__kernel": ["rbf", "linear"]
    },
    {
        "clf": [KNeighborsClassifier()],
        "clf__n_neighbors": np.arange(3, 11, 2),
        "clf__weights": ["uniform", "distance"]
    }
]

grid = GridSearchCV(pipeline, params, cv=4, verbose=3, scoring='f1')

grid.fit(X_train, y_train)